In [1]:
import numpy as np
import time

class SudokuSolver:
    def __init__(self, grid):
        self.grid = grid
        self.size = 9
        self.box_size = 3
        # Keep track of assignments for performance analysis
        self.assignments = 0
        self.backtracks = 0
    
    def is_valid(self, row, col, num):
        """Check if number can be placed at grid[row][col]"""
        # Check row
        for x in range(self.size):
            if self.grid[row][x] == num:
                return False
                
        # Check column
        for x in range(self.size):
            if self.grid[x][col] == num:
                return False
        
        # Check 3x3 box
        box_row, box_col = (row // self.box_size) * self.box_size, (col // self.box_size) * self.box_size
        for i in range(box_row, box_row + self.box_size):
            for j in range(box_col, box_col + self.box_size):
                if self.grid[i][j] == num:
                    return False
        
        return True
    
    def find_empty(self):
        """Find an empty cell in the grid"""
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j] == 0:
                    return (i, j)
        return None
    
    def find_mrv(self):
        """Find the cell with Minimum Remaining Values (MRV)"""
        min_values = float('inf')
        min_pos = None
        
        for i in range(self.size):
            for j in range(self.size):
                if self.grid[i][j] == 0:
                    # Count the number of valid values for this cell
                    num_values = sum(1 for num in range(1, 10) if self.is_valid(i, j, num))
                    if num_values < min_values:
                        min_values = num_values
                        min_pos = (i, j)
        
        return min_pos
    
    def get_lcv(self, row, col):
        """Sort values by Least Constraining Value (LCV) heuristic"""
        # For each possible value, count how many options it eliminates
        constraints = {}
        for val in range(1, 10):
            if self.is_valid(row, col, val):
                # Temporarily set the value
                self.grid[row][col] = val
                
                # Count remaining options for affected cells
                constraints_count = 0
                # Check row
                for j in range(self.size):
                    if self.grid[row][j] == 0:
                        constraints_count += sum(1 for num in range(1, 10) if self.is_valid(row, j, num))
                
                # Check column
                for i in range(self.size):
                    if self.grid[i][col] == 0:
                        constraints_count += sum(1 for num in range(1, 10) if self.is_valid(i, col, num))
                
                # Check box
                box_row, box_col = (row // self.box_size) * self.box_size, (col // self.box_size) * self.box_size
                for i in range(box_row, box_row + self.box_size):
                    for j in range(box_col, box_col + self.box_size):
                        if self.grid[i][j] == 0:
                            constraints_count += sum(1 for num in range(1, 10) if self.is_valid(i, j, num))
                
                # Reset the value
                self.grid[row][col] = 0
                constraints[val] = constraints_count
        
        # Sort values by least constraining (higher count means more options left)
        return sorted(constraints.keys(), key=lambda x: constraints[x], reverse=True)
    
    def solve_with_backtracking(self):
        """Solve sudoku using backtracking"""
        # Find an empty cell
        cell = self.find_empty()
        
        # If no empty cell, the puzzle is solved
        if cell is None:
            return True
        
        row, col = cell
        
        # Try placing digits 1-9
        for num in range(1, 10):
            # Check if valid placement
            if self.is_valid(row, col, num):
                # Place the digit
                self.grid[row][col] = num
                self.assignments += 1
                
                # Recursively try to solve the rest
                if self.solve_with_backtracking():
                    return True
                
                # If placing num doesn't lead to a solution, backtrack
                self.grid[row][col] = 0
                self.backtracks += 1
                
        # No solution found with current configuration
        return False
    
    def solve_with_mrv(self):
        """Solve sudoku using backtracking with MRV heuristic"""
        # Find cell with minimum remaining values
        cell = self.find_mrv()
        
        # If no empty cell, the puzzle is solved
        if cell is None:
            return True
        
        row, col = cell
        
        # Try placing digits 1-9
        for num in range(1, 10):
            # Check if valid placement
            if self.is_valid(row, col, num):
                # Place the digit
                self.grid[row][col] = num
                self.assignments += 1
                
                # Recursively try to solve the rest
                if self.solve_with_mrv():
                    return True
                
                # If placing num doesn't lead to a solution, backtrack
                self.grid[row][col] = 0
                self.backtracks += 1
                
        # No solution found with current configuration
        return False
    
    def solve_with_mrv_lcv(self):
        """Solve sudoku using backtracking with MRV and LCV heuristics"""
        # Find cell with minimum remaining values
        cell = self.find_mrv()
        
        # If no empty cell, the puzzle is solved
        if cell is None:
            return True
        
        row, col = cell
        
        # Get values sorted by LCV heuristic
        for num in self.get_lcv(row, col):
            # Place the digit
            self.grid[row][col] = num
            self.assignments += 1
            
            # Recursively try to solve the rest
            if self.solve_with_mrv_lcv():
                return True
            
            # If placing num doesn't lead to a solution, backtrack
            self.grid[row][col] = 0
            self.backtracks += 1
                
        # No solution found with current configuration
        return False
    
    def print_grid(self):
        """Print the grid in a readable format"""
        for i in range(self.size):
            if i % 3 == 0 and i != 0:
                print("-" * 21)
            for j in range(self.size):
                if j % 3 == 0 and j != 0:
                    print("|", end=" ")
                print(self.grid[i][j], end=" ")
            print()
    
    def get_stats(self):
        """Return statistics about the solution process"""
        return {
            "assignments": self.assignments,
            "backtracks": self.backtracks
        }

# Example usage
if __name__ == "__main__":
    # Sample puzzle from the assignment
    puzzle = [
        [0, 0, 0, 2, 6, 0, 7, 0, 1],
        [6, 8, 0, 0, 7, 0, 0, 9, 0],
        [1, 9, 0, 0, 0, 4, 5, 0, 0],
        [8, 2, 0, 1, 0, 0, 0, 4, 0],
        [0, 0, 4, 6, 0, 2, 9, 0, 0],
        [0, 5, 0, 0, 0, 3, 0, 2, 8],
        [0, 0, 9, 3, 0, 0, 0, 7, 4],
        [0, 4, 0, 0, 5, 0, 0, 3, 6],
        [7, 0, 3, 0, 1, 8, 0, 0, 0]
    ]
    
    print("Original Puzzle:")
    puzzle_array = np.array(puzzle)
    print(puzzle_array)
    print("\n")
    
    # Solve with basic backtracking
    solver1 = SudokuSolver(np.copy(puzzle_array))
    start_time = time.time()
    solver1.solve_with_backtracking()
    basic_time = time.time() - start_time
    basic_stats = solver1.get_stats()
    
    print("Solved with basic backtracking:")
    solver1.print_grid()
    print(f"Time: {basic_time:.4f} seconds")
    print(f"Assignments: {basic_stats['assignments']}")
    print(f"Backtracks: {basic_stats['backtracks']}\n")
    
    # Solve with MRV heuristic
    solver2 = SudokuSolver(np.copy(puzzle_array))
    start_time = time.time()
    solver2.solve_with_mrv()
    mrv_time = time.time() - start_time
    mrv_stats = solver2.get_stats()
    
    print("Solved with MRV heuristic:")
    solver2.print_grid()
    print(f"Time: {mrv_time:.4f} seconds")
    print(f"Assignments: {mrv_stats['assignments']}")
    print(f"Backtracks: {mrv_stats['backtracks']}\n")
    
    # Solve with MRV and LCV heuristics
    solver3 = SudokuSolver(np.copy(puzzle_array))
    start_time = time.time()
    solver3.solve_with_mrv_lcv()
    mrv_lcv_time = time.time() - start_time
    mrv_lcv_stats = solver3.get_stats()
    
    print("Solved with MRV and LCV heuristics:")
    solver3.print_grid()
    print(f"Time: {mrv_lcv_time:.4f} seconds")
    print(f"Assignments: {mrv_lcv_stats['assignments']}")
    print(f"Backtracks: {mrv_lcv_stats['backtracks']}")
    
    # Compare the performance
    print("\nPerformance Comparison:")
    print(f"Basic Backtracking: {basic_time:.4f}s, {basic_stats['assignments']} assignments, {basic_stats['backtracks']} backtracks")
    print(f"MRV Heuristic: {mrv_time:.4f}s, {mrv_stats['assignments']} assignments, {mrv_stats['backtracks']} backtracks")
    print(f"MRV+LCV Heuristics: {mrv_lcv_time:.4f}s, {mrv_lcv_stats['assignments']} assignments, {mrv_lcv_stats['backtracks']} backtracks")

Original Puzzle:
[[0 0 0 2 6 0 7 0 1]
 [6 8 0 0 7 0 0 9 0]
 [1 9 0 0 0 4 5 0 0]
 [8 2 0 1 0 0 0 4 0]
 [0 0 4 6 0 2 9 0 0]
 [0 5 0 0 0 3 0 2 8]
 [0 0 9 3 0 0 0 7 4]
 [0 4 0 0 5 0 0 3 6]
 [7 0 3 0 1 8 0 0 0]]


Solved with basic backtracking:
4 3 5 | 2 6 9 | 7 8 1 
6 8 2 | 5 7 1 | 4 9 3 
1 9 7 | 8 3 4 | 5 6 2 
---------------------
8 2 6 | 1 9 5 | 3 4 7 
3 7 4 | 6 8 2 | 9 1 5 
9 5 1 | 7 4 3 | 6 2 8 
---------------------
5 1 9 | 3 2 6 | 8 7 4 
2 4 8 | 9 5 7 | 1 3 6 
7 6 3 | 4 1 8 | 2 5 9 
Time: 0.0074 seconds
Assignments: 54
Backtracks: 9

Solved with MRV heuristic:
4 3 5 | 2 6 9 | 7 8 1 
6 8 2 | 5 7 1 | 4 9 3 
1 9 7 | 8 3 4 | 5 6 2 
---------------------
8 2 6 | 1 9 5 | 3 4 7 
3 7 4 | 6 8 2 | 9 1 5 
9 5 1 | 7 4 3 | 6 2 8 
---------------------
5 1 9 | 3 2 6 | 8 7 4 
2 4 8 | 9 5 7 | 1 3 6 
7 6 3 | 4 1 8 | 2 5 9 
Time: 0.0773 seconds
Assignments: 45
Backtracks: 0

Solved with MRV and LCV heuristics:
4 3 5 | 2 6 9 | 7 8 1 
6 8 2 | 5 7 1 | 4 9 3 
1 9 7 | 8 3 4 | 5 6 2 
---------------------